### 행안부 도로명주소 API 활용

In [1]:
import requests, json
from urllib.parse import quote          # 한글 인코딩용

In [2]:
with open('data/roadapikey.txt') as f:
    road_key = f.read()

- url 만들기

In [4]:
# 한글은 반드시 인코딩을 거쳐 송신해야함
quote('경기도청')

'%EA%B2%BD%EA%B8%B0%EB%8F%84%EC%B2%AD'

In [5]:
base_url = 'https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote("경기도청")}&resultType=json'
url=f'{base_url}?{params1}&{params2}'

- Open API를 호출하여 결과 받기

In [9]:
result = requests.get(url)
result.status_code

200

In [10]:
result.text[:200]

'({"results":{"common":{"errorMessage":"정상","countPerPage":"10","totalCount":"68","errorCode":"0","currentPage":"1"},"juso":[{"detBdNmList":"","engAddr":"30 Docheong-ro, Yeongtong-gu, Suwon-si, Gyeongg'

- JSON data를 python에서 읽을 수 있도록 변환

In [12]:
# 시작과 끝에 ()가 포함되어 오류가 발생하므로 인덱싱 등으로 없애줘야 함
res = json.loads(result.text[1:-1])
res

{'results': {'common': {'errorMessage': '정상',
   'countPerPage': '10',
   'totalCount': '68',
   'errorCode': '0',
   'currentPage': '1'},
  'juso': [{'detBdNmList': '',
    'engAddr': '30 Docheong-ro, Yeongtong-gu, Suwon-si, Gyeonggi-do',
    'rn': '도청로',
    'emdNm': '이의동',
    'zipNo': '16508',
    'roadAddrPart2': ' (이의동)',
    'emdNo': '01',
    'sggNm': '수원시 영통구',
    'jibunAddr': '경기도 수원시 영통구 이의동 0 경기도청',
    'siNm': '경기도',
    'roadAddrPart1': '경기도 수원시 영통구 도청로 30',
    'bdNm': '경기도청',
    'admCd': '4111710300',
    'udrtYn': '0',
    'lnbrMnnm': '0',
    'roadAddr': '경기도 수원시 영통구 도청로 30 (이의동)',
    'lnbrSlno': '0',
    'buldMnnm': '30',
    'bdKdcd': '0',
    'liNm': '',
    'rnMgtSn': '411173177029',
    'mtYn': '0',
    'bdMgtSn': '4111710300100000000000001',
    'buldSlno': '0'},
   {'detBdNmList': '',
    'engAddr': '28 Docheong-ro, Yeongtong-gu, Suwon-si, Gyeonggi-do',
    'rn': '도청로',
    'emdNm': '이의동',
    'zipNo': '16508',
    'roadAddrPart2': ' (이의동)',
    'emdNo': '01

- 딕셔너리 데이터에서 필요한 정보 추출하기

In [14]:
res.keys()

dict_keys(['results'])

In [16]:
res['results'].keys()

dict_keys(['common', 'juso'])

In [19]:
type(res['results']['juso'])

list

In [22]:
res['results']['juso'][0]

{'detBdNmList': '',
 'engAddr': '30 Docheong-ro, Yeongtong-gu, Suwon-si, Gyeonggi-do',
 'rn': '도청로',
 'emdNm': '이의동',
 'zipNo': '16508',
 'roadAddrPart2': ' (이의동)',
 'emdNo': '01',
 'sggNm': '수원시 영통구',
 'jibunAddr': '경기도 수원시 영통구 이의동 0 경기도청',
 'siNm': '경기도',
 'roadAddrPart1': '경기도 수원시 영통구 도청로 30',
 'bdNm': '경기도청',
 'admCd': '4111710300',
 'udrtYn': '0',
 'lnbrMnnm': '0',
 'roadAddr': '경기도 수원시 영통구 도청로 30 (이의동)',
 'lnbrSlno': '0',
 'buldMnnm': '30',
 'bdKdcd': '0',
 'liNm': '',
 'rnMgtSn': '411173177029',
 'mtYn': '0',
 'bdMgtSn': '4111710300100000000000001',
 'buldSlno': '0'}

In [23]:
# 원하는결과
res['results']['juso'][0]['roadAddr']

'경기도 수원시 영통구 도청로 30 (이의동)'

#### 경기도청, 장안구청, 권선구청, 팔달구청, 영통구청

In [25]:
gov_list = ['장안구청', '경기도청', '권선구청', '팔달구청', '영통구청']
gov_list

['장안구청', '경기도청', '권선구청', '팔달구청', '영통구청']

In [27]:
road_addr = []
for gov in gov_list:
    params2 = f'keyword={quote(gov)}&resultType=json'
    url=f'{base_url}?{params1}&{params2}'    
    result = requests.get(url)
    if result.status_code == 200:
        res = json.loads(result.text[1:-1])
        road_addr.append(res['results']['juso'][0]['roadAddr'])
    else:
        print(result.status_code)

In [28]:
road_addr

['경기도 수원시 장안구 송원로 101 (조원동)',
 '경기도 수원시 영통구 도청로 30 (이의동)',
 '경기도 수원시 권선구 호매실로 12 (탑동)',
 '경기도 수원시 팔달구 창룡대로 23 (매향동)',
 '경기도 수원시 영통구 효원로 407 (매탄동)']

In [32]:
import pandas as pd

df = pd.DataFrame({
    '이름' : gov_list, '주소' : road_addr
})
df.to_csv('data/경기도주요관공서.csv', index=False)
pd.read_csv('data/경기도주요관공서.csv')

,이름,주소
0,장안구청,경기도 수원시 장안구 송원로 101 (조원동)
1,경기도청,경기도 수원시 영통구 도청로 30 (이의동)
2,권선구청,경기도 수원시 권선구 호매실로 12 (탑동)
3,팔달구청,경기도 수원시 팔달구 창룡대로 23 (매향동)
4,영통구청,경기도 수원시 영통구 효원로 407 (매탄동)
